In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from itertools import product
from google.colab import files

In [2]:
def get_page_content(url, header, parameters):
  response = requests.get(url, headers=header, params=parameters)
  soup = BeautifulSoup(response.text, 'html.parser')
  return soup

def parse_athletes(soup):
  soup = get_page_content(URL, HEADER, PARAMETERS)
  table = soup.find('table')

  if not table:
    return None

  athletes = []
  rows = table.find_all('tr')

  if not rows:
    return None

  for row in rows:

    photo_cell = row.find('td', class_ = "photo reduced")
    name_cell = row.find('td', class_="name-academy")
    points_cell = row.find('td', class_='pontuation')
    rank_cell = row.find('td', class_='position')

    photo = photo_cell.find('img')['src']
    name_tag = name_cell.find('div', class_='name').find('a')
    name = name_tag.get_text(strip=True)
    details = DOMAIN+name_tag['href']
    points = points_cell.get_text(strip=True)
    rank = rank_cell.get_text(strip=True)

    athlete = {
      'photo':photo,
      'name':name,
      'details': details,
      'points':points,
      'rank':rank
    }

    athletes.append(athlete)
  return athletes

In [3]:
DOMAIN = 'https://ibjjf.com'
URL = f'{DOMAIN}/2025-athletes-ranking'
HEADER = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
PARAMETERS = {

  "filters[s]": "ranking-geral-gi",
  "filters[ranking_category]": "adult",
  "filters[gender]": "male",
  "filters[belt]": "black",
  "filters[weight]": "",
  "filters[search]": "",
  "commit": "Search",
  "page": 1
}

In [4]:
page = 1
all_athletes = []
while True:
  PARAMETERS['page']=page
  soup_athletes = get_page_content(URL,HEADER,PARAMETERS)
  athletes = parse_athletes(soup_athletes)
  if athletes is None:
    break
  all_athletes.extend(athletes)
  page +=1

In [5]:
df_athletes = pd.json_normalize(all_athletes)

In [6]:
df_athletes

,photo,name,details,points,rank
0,https://api.ibjjfdb.com/Athletes/134127/Rankin...,Erich Munis dos Santos,https://ibjjf.com/athletes/erich-santos,1158.0,1
1,https://api.ibjjfdb.com/Athletes/56444/Ranking...,Adam Piotr Wardzinski,https://ibjjf.com/athletes/adam-wardzinski,715.0,2
2,https://api.ibjjfdb.com/Athletes/82191/Ranking...,Diego Oliveira Batista,https://ibjjf.com/athletes/diego-batista-65fb1...,657,3
3,https://api.ibjjfdb.com/Athletes/121461/Rankin...,Marcus Vinicius Ribeiro de Siqueira,https://ibjjf.com/athletes/marcus-siqueira,605.0,4
4,https://api.ibjjfdb.com/Athletes/75845/Ranking...,Tainan Dalpra Costa,https://ibjjf.com/athletes/tainan-costa,507,5
...,...,...,...,...,...
1455,https://api.ibjjfdb.com/Athletes/200263/Rankin...,Sergio Guillermo Valbuena Piar,https://ibjjf.com/athletes/sergio-piar,1,1299
1456,https://api.ibjjfdb.com/Athletes/93769/Ranking...,Bruno da Silva,https://ibjjf.com/athletes/bruno-silva-e285a41...,1,1299
1457,https://api.ibjjfdb.com/Athletes/219568/Rankin...,Caio Santos da Silva,https://ibjjf.com/athletes/caio-silva-773253a0...,1,1299
1458,https://api.ibjjfdb.com/Athletes/86054/Ranking...,Daniel Marciano da Costa,https://ibjjf.com/athletes/daniel-costa-0e5d0d...,1,1299


In [7]:
excel_file = '/content/drive/MyDrive/web_scraping/athletes.xlsx'
df_athletes.to_excel(excel_file)